In [ ]:
!pip install langchain langchain_core langchain_groq langchain_community 

In [ ]:
!pip install langgraph

In [ ]:
import os
from typing import TypedDict , Annotated , List
from langgraph.graph import StateGraph,END 
from langchain_core.messages import HumanMessage , AIMessage
from langchain_core.prompts import ChatMessagePromptTemplate , ChatPromptTemplate
from langchain_core.runnables.graph import MermaidDrawMethod
from IPython.display import display,Image

In [ ]:
class PlanState(TypedDict):
    messages: Annotated[List[HumanMessage | AIMessage],"conversation"]
    city : str
    interest : List[str]
    itinerary : str

In [ ]:
!pip install python-dotenv


In [ ]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()

llm = ChatGroq(
    temperature = 0,
    groq_api_key = os.getenv("GROQ_API"),
    model_name = "llama-3.3-70b-versatile",
)



In [ ]:
itinerary_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant. Creae a day trip itinerary for {city} based on the user's interests: {interest}. Provide a brief, bulleted itinerary."),
    ("human","create an itinerary for my day trip")
])

In [ ]:
def cityinput(state:PlanState) -> PlanState:
    print("Please enter the city you want to visit for your day trip: ")
    user_message = input("Your Input: ")
    return {
        **state,
        "city": user_message,
        "messages": state['messages'] + [HumanMessage(content=user_message)]
    }

def interestsinput(state:PlanState) -> PlanState:
    print(f"Please enter the city: {state['city']} (comma-separted)")
    user_message = input("input: ")
    return {**state,"interest":[interest.strip() for interest in user_message.split(",")],"messages": state["messages"] + [HumanMessage(content=user_message)]}

def itinerary(state:PlanState) -> PlanState:
    print(f"Creating an itinerary for {state['city']} based on interests : {', '.join(state['interest'])}")
    response = llm.invoke(itinerary_prompt.format_messages(city = state['city'], interest = ','.join(state['interest'])))
    print("\nFinal Itinerary: ")
    print(response.content)
    return {**state,"messages": state["messages"] + [AIMessage(content=response.content)],"itinerary":response.content}

In [ ]:
workflow = StateGraph(PlanState)

workflow.add_node("cityinput",cityinput)
workflow.add_node("interestsinput",interestsinput)
workflow.add_node("itinerary",itinerary)

workflow.set_entry_point("cityinput")

workflow.add_edge("cityinput","interestsinput")
workflow.add_edge("interestsinput","itinerary")
workflow.add_edge("itinerary",END)

app = workflow.compile()

In [ ]:
display(
    Image(app.get_graph().draw_mermaid_png(
        draw_method=MermaidDrawMethod.API
    ))
)

In [ ]:
def travel(city: str, interest: str):

    print("intial")

    state = {
        "messages":"",
        "city":"",
        "interest":[],
        "itinerary":"",
    }

    state = cityinput(city, state)
    state = interestsinput(interest, state)

    itinerary_res = itinerary(state)

    return itinerary_res


In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
interface = gr.Interface(
    fn=travel,
    theme='Yntec/HaleyCH_Theme_Orange_Green',
    inputs=[
        gr.Textbox(label="Enter the city for your day trip"),
        gr.Textbox(label="Enter your interests (comma-separated)"),
    ],
    outputs=gr.Textbox(label="Generated Itinerary"),
    title="Travel Itinerary Planner",
    description="Enter a city and your interests to generate a personalized day trip itinerary."
)

interface.launch()


In [48]:
import gradio as gr
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()

class PlannerState(TypedDict):
    messages: Annotated[List[HumanMessage | AIMessage], "The messages in the conversation"]
    city: str
    interests: List[str]
    itinerary: str

llm = ChatGroq(
    temperature=0,
    groq_api_key=os.getenv("GROQ_API"),
    model_name="llama-3.3-70b-versatile"
)

itinerary_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful travel assistant. Create a day trip itinerary for {city} based on the user's interests: {interests}. Provide a brief, bulleted itinerary."),
    ("human", "Create an itinerary for my day trip."),
])

def input_city(city: str, state: PlannerState) -> PlannerState:
    return {
        **state,
        "city": city,
        "messages": state['messages'] + [HumanMessage(content=city)],
    }

def input_interests(interests: str, state: PlannerState) -> PlannerState:
    return {
        **state,
        "interests": [interest.strip() for interest in interests.split(',')],
        "messages": state['messages'] + [HumanMessage(content=interests)],
    }

def create_itinerary(state: PlannerState) -> str:
    response = llm.invoke(itinerary_prompt.format_messages(city=state['city'], interests=", ".join(state['interests'])))
    state["itinerary"] = response.content
    state["messages"] += [AIMessage(content=response.content)]
    return response.content

def travel_planner(city: str, interests: str):
    state = {
        "messages": [],
        "city": "",
        "interests": [],
        "itinerary": "",
    }

    state = input_city(city, state)
    state = input_interests(interests, state)

    itinerary = create_itinerary(state)

    return itinerary

interface = gr.Interface(
    fn=travel_planner,
    theme='Yntec/HaleyCH_Theme_Orange_Green',
    inputs=[
        gr.Textbox(label="Enter the city for your day trip"),
        gr.Textbox(label="Enter your interests (comma-separated)"),
    ],
    outputs=gr.Textbox(label="Generated Itinerary"),
    title="Travel Itinerary Planner",
    description="Enter a city and your interests to generate a personalized day trip itinerary."
)

interface.launch()


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
